# FINN model export + estimates

This notebook builds a Brevitas INT8 model, optionally loads a checkpoint, exports QONNX, and provides placeholders for FINN estimate steps.

In [1]:
import os
import torch
import torch.nn as nn

import brevitas.nn as qnn
from brevitas.quant import Int8ActPerTensorFixedPoint, Int8WeightPerTensorFixedPoint
from brevitas.onnx import export_qonnx
from qonnx.util.cleanup import cleanup as qonnx_cleanup
from finn.util.visualization import showSrc , showInNetron
from torch.utils.data import Dataset, DataLoader, random_split
from PIL import Image
from tqdm import tqdm
import torchvision.transforms as T
from pathlib import Path
import shutil
import inspect
import finn.builder.build_dataflow as build
import finn.builder.build_dataflow_config as build_cfg

In [3]:
pth_name = "best_map.pth"

onnx_name = "yolo_split_backbone.onnx"
onnx_finn_name = "yolo_split_backbone.onnx"
build_dir = f"/home/ug_lab/finn/trained_result"

export_path_onnx = f"/home/ug_lab/finn/trained_result/{onnx_name}"
export_path_onnx_finn = f"/home/ug_lab/finn/notebooks/pls11/{onnx_finn_name}"
pth_path = f"/home/ug_lab/finn/trained_result/{pth_name}"

# folding_json = f"/mnt/c/Users/bsuth/finn/notebooks/poz_train/custom_folding.json"

export_path_final_reports = f"/home/ug_lab/finn/notebooks/pls11/final_output_reports/{onnx_name}_reports"
model_file = export_path_onnx_finn
estimates_output = f"{export_path_final_reports}/output_estimates"
estimates_output_dir = f"{export_path_final_reports}/{onnx_name}_reports_v2"


In [3]:
import torch
import torch.nn as nn
import brevitas.nn as qnn
from brevitas.nn import QuantConv2d, QuantReLU, QuantIdentity
from brevitas.quant import Int8WeightPerTensorFloat as WeightQuant
from brevitas.quant import Int8ActPerTensorFloat as ActQuant
from brevitas.core.restrict_val import RestrictValueType

# Optimized Quantizer for FINN Hardware
class FinnWeightQuant(WeightQuant):
    restrict_scaling_type = RestrictValueType.POWER_OF_TWO

class FinnActQuant(ActQuant):
    restrict_scaling_type = RestrictValueType.POWER_OF_TWO
    signed = False
    narrow_range = False
    min_val = 0.0

class TinyissimoYOLO_Standard(nn.Module):
    def __init__(self, num_classes=1):
        super().__init__()

        # FINN expects QuantIdentity activations to be signed
        self.input_quant = QuantIdentity(
            act_quant=ActQuant,
            return_quant_tensor=False,
            signed=True,
            narrow_range=False,
        )

        # Layer 1: Stem (1 -> 32)
        self.conv1 = QuantConv2d(1, 32, kernel_size=3, stride=2, padding=1, bias=False, weight_quant=FinnWeightQuant)
        self.bn1 = nn.BatchNorm2d(32)
        self.act1 = QuantReLU(
            bit_width=8,
            act_quant=FinnActQuant,
            return_quant_tensor=False,
            signed=False,
            narrow_range=False,
        )

        # Main Body - Adjusted channels to fit < 600K
        self.layer2 = self._make_layer(32, 64, stride=2)   # 18K
        self.layer3 = self._make_layer(64, 112, stride=2)  # 64K (Reduced from 128)
        self.layer4 = self._make_layer(112, 128, stride=2) # 129K
        self.layer5 = self._make_layer(128, 144, stride=1) # 165K (Reduced from 160)
        self.layer6 = self._make_layer(144, 144, stride=1) # 186K (Reduced from 160)

        # Head: 144 -> 6 (Obj, x, y, w, h, Class)
        self.final_conv = QuantConv2d(144, (5 + num_classes), kernel_size=1, bias=False, weight_quant=FinnWeightQuant)

    def _make_layer(self, in_c, out_c, stride):
        return nn.Sequential(
            QuantConv2d(in_c, out_c, kernel_size=3, stride=stride, padding=1, bias=False, weight_quant=FinnWeightQuant),
            nn.BatchNorm2d(out_c),
            QuantReLU(
                bit_width=4,
                act_quant=FinnActQuant,
                return_quant_tensor=False,
                signed=False,
                narrow_range=False,
            ),
        )

    def forward(self, x):
        x = self.input_quant(x)
        x = self.act1(self.bn1(self.conv1(x)))
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x = self.layer6(x)
        return self.final_conv(x)

# --- 2. Instantiate & Patch ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TinyissimoYOLO_Standard(num_classes=1).to(device)

def ultimate_patch(model):
    for m in model.modules():
        for attr in ['weight_quant_proxy', 'act_quant_proxy', 'input_quant_proxy', 'bias_quant_proxy']:
            if hasattr(m, attr):
                setattr(getattr(m, attr), 'requires_input_scale', False)

ultimate_patch(model)

# --- 3. Parameter Verification ---
total_params = sum(p.numel() for p in model.parameters())
print(f"{'='*40}")
print(f"✅ Model: Standard QuantConv2d (FINN-Ready)")
print(f"📊 Total Parameters: {total_params:,}")
print(f"🚀 Status: {'UNDER LIMIT' if total_params < 600000 else 'OVER LIMIT'}")
print(f"{'='*40}")

✅ Model: Standard QuantConv2d (FINN-Ready)
📊 Total Parameters: 566,887
🚀 Status: UNDER LIMIT


In [4]:
model_name = TinyissimoYOLO_Standard
CHECKPOINT_PATH = pth_path

model = model_name()
if CHECKPOINT_PATH and os.path.exists(CHECKPOINT_PATH):
    state = torch.load(CHECKPOINT_PATH, map_location="cpu")
    model.load_state_dict(state, strict=False)
    print(f"Loaded checkpoint: {CHECKPOINT_PATH}")

model.eval()
model

dummy_inp = torch.randn(1, 1, 128, 128)
export_qonnx(model, dummy_inp, str(export_path_onnx))

# if not export_path_onnx.exists():
#     raise FileNotFoundError(f"QONNX export failed, file not found at: {export_path_onnx}")

print(f"Saved QONNX to: {export_path_onnx}")

Loaded checkpoint: /home/ug_lab/finn/trained_result/best_map.pth
Saved QONNX to: /home/ug_lab/finn/trained_result/best_map_v2.onnx


All Locations

In [5]:
from pathlib import Path
from qonnx.core.modelwrapper import ModelWrapper
from qonnx.transformation.infer_shapes import InferShapes
from qonnx.transformation.fold_constants import FoldConstants
from qonnx.transformation.general import GiveReadableTensorNames, GiveUniqueNodeNames, RemoveStaticGraphInputs
from qonnx.transformation.infer_datatypes import InferDataTypes
from qonnx.util.cleanup import cleanup_model

from finn.transformation.qonnx.convert_qonnx_to_finn import ConvertQONNXtoFINN
from finn.transformation.streamline import Streamline
from finn.transformation.streamline.round_thresholds import RoundAndClipThresholds

model = ModelWrapper(export_path_onnx)
model = cleanup_model(model)

model = model.transform(InferShapes())
model = model.transform(FoldConstants())
model = model.transform(GiveUniqueNodeNames())
model = model.transform(GiveReadableTensorNames())
model = model.transform(RemoveStaticGraphInputs())
model = cleanup_model(model)

# Convert QONNX quant nodes into FINN dialect first
model = model.transform(ConvertQONNXtoFINN())
model = cleanup_model(model)

# Then apply FINN streamlining passes
model = model.transform(Streamline())
model = model.transform(RoundAndClipThresholds())
model = model.transform(InferDataTypes())
model = cleanup_model(model)

Path(export_path_onnx_finn).parent.mkdir(parents=True, exist_ok=True)
model.save(export_path_onnx_finn)
print("✅ Converted and saved FINN model:", export_path_onnx_finn)

✅ Converted and saved FINN model: /home/ug_lab/finn/notebooks/pls11/best_map_finn_v2.onnx


In [ ]:
!pip install opencv-python

In [7]:
import os
import torch
import numpy as np
import cv2

# Import your Brevitas model definition
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Create model
model = TinyissimoYOLO_Standard().to(device)
checkpoint = torch.load(pth_path, map_location=device)

def _strip_prefixes(state_dict, prefixes):
    out = {}
    for key, value in state_dict.items():
        new_key = key
        changed = True
        while changed:
            changed = False
            for prefix in prefixes:
                if new_key.startswith(prefix):
                    new_key = new_key[len(prefix):]
                    changed = True
        out[new_key] = value
    return out

def _best_state_dict_for_model(raw_state, model_keys):
    candidates = []
    if isinstance(raw_state, dict):
        candidates.append(raw_state)
        candidates.append(_strip_prefixes(raw_state, ["module."]))
        candidates.append(_strip_prefixes(raw_state, ["model."]))
        candidates.append(_strip_prefixes(raw_state, ["module.", "model.", "net."]))

    best_sd = None
    best_overlap = -1
    for cand in candidates:
        overlap = len(set(cand.keys()) & model_keys)
        if overlap > best_overlap:
            best_overlap = overlap
            best_sd = cand
    return best_sd, best_overlap

# Support multiple checkpoint formats
if isinstance(checkpoint, dict) and "model_state_dict" in checkpoint:
    raw_state = checkpoint["model_state_dict"]
elif isinstance(checkpoint, dict) and "state_dict" in checkpoint:
    raw_state = checkpoint["state_dict"]
else:
    raw_state = checkpoint

if not isinstance(raw_state, dict):
    raise TypeError("Checkpoint does not contain a valid state_dict.")

model_keys = set(model.state_dict().keys())
state_dict, overlap = _best_state_dict_for_model(raw_state, model_keys)
match_ratio = overlap / max(1, len(model_keys))

if match_ratio < 0.70:
    raise RuntimeError(
        f"Checkpoint key match too low: {match_ratio:.1%}. "
        "Likely wrong checkpoint format/model definition."
    )

missing_keys, unexpected_keys = model.load_state_dict(state_dict, strict=False)
model.eval()

print(f"✅ Checkpoint key match: {match_ratio:.1%}")
if missing_keys:
    print(f"⚠️ Missing keys: {len(missing_keys)}")
if unexpected_keys:
    print(f"⚠️ Unexpected keys: {len(unexpected_keys)}")

def preprocess_image(image_path, size=(128, 128)):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    
    if img is None:
        raise ValueError("Image not found")
    
    img = cv2.resize(img, size, interpolation=cv2.INTER_AREA)
    
    # IMPORTANT: normalize to 0..1 for current quantizer setup
    img = img.astype(np.float32)
    
    # Add batch + channel → (1,1,128,128)
    img = np.expand_dims(img, axis=0)
    img = np.expand_dims(img, axis=0)

    tensor = torch.from_numpy(img).float()
    
    return tensor


image_path = "/home/ug_lab/finn/notebooks/yolo_finn/Gao_ship_hh_02016082544040403.jpg"

input_tensor = preprocess_image(image_path)/255
input_tensor = input_tensor.to(device)/255

with torch.no_grad():
    output = model(input_tensor)

# Move to numpy
input_np = input_tensor.detach().cpu().numpy().astype(np.float32)
output_np = output.detach().cpu().numpy().astype(np.float32)

# Save output (legacy file)
np.save("output_from_pth.npy", output_np)

# Save FINN verification files
verify_dir = "/home/ug_lab/finn/notebooks/yolo_finn"
os.makedirs(verify_dir, exist_ok=True)
verify_input_path = os.path.join(verify_dir, "input2.npy")
verify_output_path = os.path.join(verify_dir, "output2.npy")
np.save(verify_input_path, input_np)
np.save(verify_output_path, output_np)

print("Inference done")
print("Input shape:", input_np.shape)
print("Input sample (flattened):", input_np.flatten()[:1000])

print("Output shape:", output_np.shape)
print(f"✅ Saved verify input: {verify_input_path}")
print(f"✅ Saved verify output: {verify_output_path}")
print("Output sample (flattened):", output_np.flatten()[:10])


✅ Checkpoint key match: 100.0%
Inference done
Input shape: (1, 1, 128, 128)
Input sample (flattened): [0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 4.6136101e-05 3.0757405e-05 4.6136101e-05
 7.6893506e-05 6.1514809e-05 1.2302962e-04 2.4605924e-04 4.1522493e-04
 3.8446751e-04 1.2149174e-03 1.2302962e-04 3.0

In [8]:
from finn.util.pytorch import ToTensor, NormalizePreProc
from qonnx.transformation.merge_onnx_models import MergeONNXModels
from qonnx.core.datatype import DataType
from qonnx.core.modelwrapper import ModelWrapper
from brevitas.onnx import export_qonnx

# Load FINN ONNX model wrapper explicitly (do not reuse torch model variable)
finn_model = ModelWrapper(export_path_onnx_finn)
global_inp_name = finn_model.graph.input[0].name
print("Global input name:", global_inp_name)
ishape = torch.randn(1, 1, 128, 128)

print(f"Preproc export input shape: {tuple(ishape.shape)}")

# preprocessing: torchvision's ToTensor divides uint8 inputs by 255
totensor_pyt = ToTensor()

chkpt_preproc_name = build_dir + "/best_map.onnx"
export_qonnx(totensor_pyt, ishape, chkpt_preproc_name)

# join preprocessing and core model
pre_model = ModelWrapper(chkpt_preproc_name)
finn_model = finn_model.transform(MergeONNXModels(pre_model))

# add input quantization annotation: UINT8 for BNN-PYNQ models
global_inp_name = finn_model.graph.input[0].name
finn_model.set_tensor_datatype(global_inp_name, DataType["UINT8"])
finn_model.save(export_path_onnx_finn)
print("✅ Merged preproc and saved:", export_path_onnx_finn)

# showInNetron(export_path_onnx_finn)

Global input name: global_in
Preproc export input shape: (1, 1, 128, 128)
✅ Merged preproc and saved: /home/ug_lab/finn/notebooks/pls11/best_map_finn_v2.onnx


/home/ug_lab/finn/deps/qonnx/src/qonnx/transformation/infer_data_layouts.py:127: UserWarning: Assuming 4D input is NCHW
  warnings.warn("Assuming 4D input is NCHW")


In [9]:
from qonnx.core.modelwrapper import ModelWrapper
from qonnx.transformation.infer_shapes import InferShapes
from qonnx.transformation.fold_constants import FoldConstants
from qonnx.transformation.general import (
    GiveReadableTensorNames,
    GiveUniqueNodeNames,
    RemoveStaticGraphInputs,
)
from qonnx.util.cleanup import cleanup_model

from finn.transformation.qonnx.convert_qonnx_to_finn import ConvertQONNXtoFINN
from finn.transformation.streamline import Streamline
from finn.transformation.streamline.round_thresholds import RoundAndClipThresholds
from finn.transformation.streamline.absorb import (
    AbsorbAddIntoMultiThreshold,
    AbsorbMulIntoMultiThreshold,
)
from finn.transformation.streamline.reorder import (
    MoveMulPastMaxPool,
    MoveAddPastMul,
)
from qonnx.transformation.infer_datatypes import InferDataTypes

# Always load ONNX as ModelWrapper here (avoid torch model shadowing)
model = ModelWrapper(export_path_onnx_finn)
model = cleanup_model(model)
model = model.transform(InferShapes())
model = model.transform(FoldConstants())
model = model.transform(GiveUniqueNodeNames())
model = model.transform(GiveReadableTensorNames())
model = model.transform(RemoveStaticGraphInputs())
model = cleanup_model(model)

# Convert only if still in QONNX dialect
try:
    model = model.transform(ConvertQONNXtoFINN())
    model = cleanup_model(model)
except Exception as e:
    print("ConvertQONNXtoFINN skipped/failed (possibly already FINN dialect):", type(e).__name__)

model = model.transform(Streamline())
model = model.transform(AbsorbAddIntoMultiThreshold())
model = model.transform(AbsorbMulIntoMultiThreshold())
model = model.transform(MoveMulPastMaxPool())
model = model.transform(MoveAddPastMul())
model = model.transform(RoundAndClipThresholds())
model = model.transform(InferDataTypes())

model.save(export_path_onnx_finn)
print("✅ FINN-ready model saved.")

✅ FINN-ready model saved.


In [ ]:

# Delete previous run results if they exist.
if os.path.exists(estimates_output):
    shutil.rmtree(estimates_output )
    print("Previous run results deleted.")

if not os.path.exists(model_file):
    raise FileNotFoundError(f"Model file not found: {model_file}")

cfg_estimates = build.DataflowBuildConfig(
    output_dir = estimates_output,
    # mvau_wwidth_max = 120,
    synth_clk_period_ns = 50.0,
    split_large_fifos=True,
    target_fps=60,   # LOWER
    # folding_config_file = folding_json,
    auto_fifo_depths=True,
    fpga_part = "xc7z020clg400-1",
    board = "Pynq-Z2",
    steps = build_cfg.default_build_dataflow_steps,
    # verify_steps = [build_cfg.VerificationStepType.QONNX_TO_FINN_PYTHON,
    #                 # build_cfg.VerificationStepType.STREAMLINED_PYTHON,
    #                 # build_cfg.VerificationStepType.TIDY_UP_PYTHON,
    #                 build_cfg.VerificationStepType.FOLDED_HLS_CPPSIM,
    #                 build_cfg.VerificationStepType.STITCHED_IP_RTLSIM],
    # verify_input_npy = "/home/ug_lab/finn/notebooks/yolo_finn/input2.npy",
    # verify_expected_output_npy = "/home/ug_lab/finn/notebooks/yolo_finn/output2.npy",
    # verify_save_full_context = True,  
    
    generate_outputs=[
        build_cfg.DataflowOutputType.ESTIMATE_REPORTS,
        build_cfg.DataflowOutputType.ESTIMATE_REPORTS,
        build_cfg.DataflowOutputType.STITCHED_IP,
        build_cfg.DataflowOutputType.RTLSIM_PERFORMANCE
        # build_cfg.DataflowOutputType.BITFILE,
        # build_cfg.DataflowOutputType.PYNQ_DRIVER,
        # build_cfg.DataflowOutputType.OOC_SYNTH,   
    ]
)

build.build_dataflow_cfg(model_file, cfg_estimates)

! cat {estimates_output}/report/estimate_network_performance.json

import json
def read_json_dict(filename):
    with open(filename, "r") as f:
        ret = json.load(f)
    return ret

read_json_dict(estimates_output + "/report/estimate_layer_cycles.json")
read_json_dict(estimates_output + "/report/estimate_layer_resources.json")

estimates report

In [11]:
res = read_json_dict(estimates_output + "/report/estimate_layer_resources.json")
tot = res.get("total", {})

# XC7Z020 (PYNQ-Z2) resource totals from ds187.
TOTAL_LUT = 53200
TOTAL_FF = 106400
TOTAL_BRAM36 = 140
TOTAL_DSP = 220

# FINN reports BRAM_18K; convert to BRAM36K blocks.
bram18 = float(tot.get("BRAM_18K", 0))
bram36 = bram18 / 2.0
lut = float(tot.get("LUT", 0))
dsp = float(tot.get("DSP", 0))
ff = float(tot.get("FF", tot.get("REG", tot.get("Register", 0))))

def pct(used, total):
    return 0.0 if total == 0 else (used / total) * 100.0

util = {
    "LUT_used": lut,
    "LUT_pct": pct(lut, TOTAL_LUT),
    "FF_used": ff,
    "FF_pct": pct(ff, TOTAL_FF),
    "BRAM36_used": bram36,
    "BRAM36_pct": pct(bram36, TOTAL_BRAM36),
    "DSP_used": dsp,
    "DSP_pct": pct(dsp, TOTAL_DSP),
}

print(util)

{'LUT_used': 26722.0, 'LUT_pct': 50.22932330827068, 'FF_used': 0.0, 'FF_pct': 0.0, 'BRAM36_used': 157.0, 'BRAM36_pct': 112.14285714285714, 'DSP_used': 1.0, 'DSP_pct': 0.45454545454545453}


In [ ]:
import numpy as np
from qonnx.core.modelwrapper import ModelWrapper
from qonnx.core.onnx_exec import execute_onnx
from qonnx.core.datatype import DataType
import time

model_path = "/mnt/c/Users/bsuth/finn/notebooks/yolo_finn/best.finn.onnx_finn.onnx"
model = ModelWrapper(model_path)

# First input
inp_name = model.graph.input[0].name
inp_shape = model.get_tensor_shape(inp_name)
inp_dt = model.get_tensor_datatype(inp_name) or DataType.FLOAT32

def _np_dtype(dt):
    if hasattr(dt, "numpy"):
        return dt.numpy()
    dt_str = str(dt).lower()
    if "float16" in dt_str:
        return np.float16
    if "float" in dt_str:
        return np.float32
    if "uint8" in dt_str:
        return np.uint8
    if "int8" in dt_str:
        return np.int8
    if "uint16" in dt_str:
        return np.uint16
    if "int16" in dt_str:
        return np.int16
    if "uint32" in dt_str:
        return np.uint32
    if "int32" in dt_str:
        return np.int32
    if "uint64" in dt_str:
        return np.uint64
    if "int64" in dt_str:
        return np.int64
    return np.float32

def _is_integer(dt, np_dt):
    if hasattr(dt, "is_integer"):
        return dt.is_integer()
    return np.issubdtype(np_dt, np.integer)

def _int_range(dt, np_dt):
    if hasattr(dt, "min") and hasattr(dt, "max"):
        return dt.min(), dt.max()
    info = np.iinfo(np_dt)
    return info.min, info.max

np_dt = _np_dtype(inp_dt)

# Random input matching dtype/range
if _is_integer(inp_dt, np_dt):
    lo, hi = _int_range(inp_dt, np_dt)
    x = np.random.randint(lo, hi + 1, size=inp_shape, dtype=np_dt)
else:
    x = np.random.randn(*inp_shape).astype(np_dt)

np.save("/mnt/c/Users/bsuth/finn/notebooks/yolo_finn/input.npy", x)

# Run
start_time = time.time()

outputs = execute_onnx(model, {inp_name: x})
end_time = time.time()
print(f"Execution time: {end_time - start_time} seconds")

out0 = next(iter(outputs.values()))
np.save("/mnt/c/Users/bsuth/finn/notebooks/yolo_finn/output.npy", out0)

for name, val in outputs.items():
    print(name, val.shape)
    print(val)

In [ ]:
import numpy as np
from qonnx.core.modelwrapper import ModelWrapper
from qonnx.core.datatype import DataType

model = ModelWrapper("/mnt/c/Users/bsuth/finn/notebooks/yolo_finn/best.finn.onnx_finn.onnx")
inp_name = model.graph.input[0].name
out_name = model.graph.output[0].name
inp_shape = model.get_tensor_shape(inp_name)
out_shape = model.get_tensor_shape(out_name)
inp_dt = model.get_tensor_datatype(inp_name) or DataType.FLOAT32

x = np.load("/mnt/c/Users/bsuth/finn/notebooks/yolo_finn/input.npy")
y = np.load("/mnt/c/Users/bsuth/finn/notebooks/yolo_finn/output.npy")

print("model input shape:", inp_shape, "dtype:", inp_dt)
print("model output shape:", out_shape)
print("input.npy shape:", x.shape, "dtype:", x.dtype)
print("output.npy shape:", y.shape, "dtype:", y.dtype)

model input shape: [1, 3, 416, 416] dtype: FLOAT32
model output shape: [1, 18, 13, 13]
input.npy shape: (1, 3, 416, 416) dtype: float32
output.npy shape: (1, 18, 13, 13) dtype: float32


## Verification debug summary

This cell summarizes PASS/FAIL and compares FAIL outputs against expected.

In [ ]:
import os
import numpy as np

base_reports_dir = "/mnt/c/Users/bsuth/finn/notebooks/yolo_finn/final_output_reports"
ver_dir = f"{base_reports_dir}/{onnx_name}_reports/output_estimates/verification_output"
expected_path = "/mnt/c/Users/bsuth/finn/notebooks/yolo_finn/output.npy"
input_path = "/mnt/c/Users/bsuth/finn/notebooks/yolo_finn/input.npy"

if not os.path.isdir(ver_dir):
    raise FileNotFoundError(f"verification_output not found: {ver_dir}")

npz_files = sorted(
    [os.path.join(ver_dir, f) for f in os.listdir(ver_dir) if f.endswith(".npz")],
    key=lambda p: os.path.getmtime(p),
)
pass_files = [p for p in npz_files if "_PASS" in os.path.basename(p)]
fail_files = [p for p in npz_files if "_FAIL" in os.path.basename(p)]

print("verification_output:", ver_dir)
print("PASS files:", len(pass_files))
print("FAIL files:", len(fail_files))
if pass_files:
    print("latest PASS:", os.path.basename(pass_files[-1]))
if fail_files:
    print("latest FAIL:", os.path.basename(fail_files[-1]))

if not fail_files:
    print("No FAIL files found.")
else:
    fail_path = fail_files[-1]
    data = np.load(fail_path)
    print("using FAIL:", os.path.basename(fail_path))
    print("keys:", list(data.keys()))

    np.set_printoptions(threshold=200, edgeitems=3, linewidth=120)
    exp = np.load(expected_path)
    print("expected output:", exp)

    if "global_out" in data:
        print("global_out:", data["global_out"])
    else:
        match_keys = [k for k in data.keys() if data[k].shape == exp.shape]
        if match_keys:
            out_key = match_keys[0]
            print("global_out (matched by shape):", data[out_key])
        else:
            print("global_out not found and no matching shape.")

    match_keys = [k for k in data.keys() if data[k].shape == exp.shape]
    if not match_keys:
        print("No tensor matches expected output shape.")
    else:
        out_key = match_keys[0]
        out = data[out_key]
        diff = np.abs(exp - out)
        print("output key:", out_key)
        print("max abs diff:", float(diff.max()))
        print("mean abs diff:", float(diff.mean()))
        idx = np.unravel_index(np.argmax(diff), diff.shape)
        print("worst idx:", idx, "expected:", exp[idx], "actual:", out[idx])

    if "global_in" in data and os.path.exists(input_path):
        ver_in = data["global_in"]
        inp = np.load(input_path)
        if ver_in.shape != inp.shape:
            print("global_in shape differs from input.npy")
            print("global_in:", ver_in.shape, "input.npy:", inp.shape)
        else:
            in_diff = np.abs(ver_in - inp)
            print("global_in vs input.npy max abs diff:", float(in_diff.max()))
            print("global_in vs input.npy mean abs diff:", float(in_diff.mean()))
    else:
        print("global_in not found in npz or input.npy missing.")

verification_output: /mnt/c/Users/bsuth/finn/notebooks/yolo_finn/final_output_reports/best.finn.onnx_reports/output_estimates/verification_output
PASS files: 0
FAIL files: 2
latest FAIL: verify_stitched_ip_rtlsim_0_FAIL.npz
using FAIL: verify_stitched_ip_rtlsim_0_FAIL.npz
keys: ['global_in', 'global_out', 'Mul_0_param0', 'Transpose_1_out0', 'MultiThreshold_0_param0', 'MatMul_0_param0', 'Transpose_0_out0', 'Im2Col_0_out0', 'MatMul_0_out0', 'MultiThreshold_0_out0', 'MultiThreshold_9_out0', 'GenericPartition_0_global_in', 'GenericPartition_0_MVAU_hls_0_param1', 'GenericPartition_0_MVAU_hls_1_param1', 'GenericPartition_0_MVAU_hls_2_param1', 'GenericPartition_0_MVAU_hls_3_param1', 'GenericPartition_0_MVAU_hls_4_param1', 'GenericPartition_0_MVAU_hls_5_param1', 'GenericPartition_0_MVAU_hls_6_param1', 'GenericPartition_0_MVAU_hls_7_param1', 'GenericPartition_0_MVAU_hls_8_param1', 'GenericPartition_0_MVAU_hls_0_param0', 'GenericPartition_0_MVAU_hls_1_param0', 'GenericPartition_0_MVAU_hls_2_para

FLOW FOR GENERATING THE HLS CODES


In [ ]:


# Delete previous run results if they exist.
if os.path.exists(estimates_output_dir):
    shutil.rmtree(estimates_output_dir)
    print("Previous run results deleted.")

if not os.path.exists(model_file):
    raise FileNotFoundError(f"Model file not found: {model_file}")

# Build config kwargs, filtered to what this FINN version accepts.
# cfg_kwargs = {
#     "output_dir": estimates_output_dir,
#     "mvau_wwidth_max": 80,
#     "target_fps": 3000,
#     "synth_clk_period_ns": 10,
#     "fpga_part": "xc7z020clg400-1",
#     "board": "Pynq-Z2",
#     "shell_flow_type": build_cfg.ShellFlowType.VIVADO_ZYNQ,
#     "steps": build_cfg.default_build_dataflow_steps,
#     "generate_outputs":[
#         build_cfg.DataflowOutputType.STITCHED_IP,
#         build_cfg.DataflowOutputType.ESTIMATE_REPORTS,
#         build_cfg.DataflowOutputType.BITFILE,
#         build_cfg.DataflowOutputType.DEPLOYMENT_PACKAGE,
#         build_cfg.DataflowOutputType.OOC_SYNTH,
#         build_cfg.DataflowOutputType.PYNQ_DRIVER,
#         build_cfg.DataflowOutputType.RTLSIM_PERFORMANCE
#         build_cfg.DataflowOutputType.
#     ]
#     # "steps": build_cfg.estimate_only_dataflow_steps 
# # }

# sig = inspect.signature(build_cfg.DataflowBuildConfig)
# print(f"DataflowBuildConfig parameters: {sig.parameters.keys()}")
# filtered_kwargs = {k: v for k, v in cfg_kwargs.items() if k in sig.parameters}

# # Prefer estimate reports output type if available, else use string token.
# if "generate_outputs" in sig.parameters:
#     output_type = getattr(build_cfg, "DataflowOutputType", None)
#     if output_type is not None and hasattr(output_type, "ESTIMATE_REPORTS"):
#         filtered_kwargs["generate_outputs"] = [output_type.ESTIMATE_REPORTS]
#     else:
#         filtered_kwargs["generate_outputs"] = ["estimate"]

# cfg = build_cfg.DataflowBuildConfig(**filtered_kwargs)

# def resolve_build_fn():
#     try:
#         from finn.builder.build_dataflow import build_dataflow as direct_build_dataflow
#         return direct_build_dataflow
#     except Exception:
#         pass
#     candidates = ["build_dataflow", "build_dataflow_cfg", "build_dataflow_app"]
#     for name in candidates:
#         fn = getattr(build, name, None)
#         if callable(fn):
#             return fn
#     for name in candidates:
#         fn = getattr(build_cfg, name, None)
#         if callable(fn):
#             return fn
#     return None

# # Run FINN build to generate analytic + HLS estimates.
# build_fn = resolve_build_fn()
# if build_fn is None:
#     available = [name for name, obj in inspect.getmembers(build) if callable(obj)]
#     raise AttributeError(
#         "No build_dataflow function found in this FINN version. "
#         f"Callable names in finn.builder.build_dataflow: {available}"
# )
# build_fn(model_file, cfg)

# print(f"Reports are in: {estimates_output_dir}/reports")

cfg = build.DataflowBuildConfig(
    output_dir          = estimates_output,
    mvau_wwidth_max = 36,
    synth_clk_period_ns = 20.0,
    split_large_fifos=True,
    target_fps=1000,   # LOWER
    folding_config_file = folding_json,
    auto_fifo_depths=False,   
    fpga_part           = "xc7z020clg400-1",
    board = "Pynq-Z2",
    generate_outputs=[
        build_cfg.DataflowOutputType.ESTIMATE_REPORTS,
        build_cfg.DataflowOutputType.ESTIMATE_REPORTS,
        build_cfg.DataflowOutputType.STITCHED_IP,
        build_cfg.DataflowOutputType.RTLSIM_PERFORMANCE,
        build_cfg.DataflowOutputType.BITFILE,
        build_cfg.DataflowOutputType.PYNQ_DRIVER,
        build_cfg.DataflowOutputType.OOC_SYNTH,
        
    ]
)



In [ ]:
print(model.graph.node)


In [ ]:
%%time
build.build_dataflow_cfg(model_file, cfg)

In [ ]:
from PIL import Image
import numpy as np

img_path = "/mnt/c/Users/bsuth/finn/notebooks/poz_train/dataset/Dataset_for_SAR_images/images/Gao_ship_hh_02016082544040403.jpg"
image = Image.open(img_path).convert("RGB")
image = image.resize((128, 128))
w, h = image.size
image = T.ToTensor()(image)
print(image.shape, image.dtype)